In [1]:
import pandas as pd
data2 = pd.read_excel('SearchItems.xlsx', sheet_name = 'Sheet1', header = 0, encoding = 'ISO-8859-1')

In [23]:
data2.columns = ['Term', 'Frequence']
data2.head()

,Term,Frequence
0,Pin điện thoại iPhone 6,123
1,điện thoại samsung galaxy a8 plus,121
2,Đồng hồ định vị GPS (kiêm điện thoại trẻ em) I...,83
3,điện thoại cho người già,78
4,điện thoại iphone 6 plus,67


In [102]:
from underthesea import word_tokenize
dic = []
for item in data2.Term.tolist():
    dic.append(word_tokenize(item))

In [106]:
singular_word = list(map(str.split, data2.Term))
dic0 = []
for item in singular_word:
    dic0 += item
# dic1

In [104]:
dic1 = []
for i in range(len(dic)):
    dic1 += dic[i]
print('Length of the dictionary: ', len(dic1))

Length of the dictionary:  32923


In [108]:
dic2 = dic0 + dic1

['Gimbal', 'chống', 'rung', 'cho', 'điện thoại']

In [5]:
import numpy as np
dic3 = np.unique(dic2)
print('Length of the dictionary: ', len(dic3))

Length of the dictionary:  3701


In [7]:
X = np.unique(dic2, return_counts = True)
data_tf = pd.DataFrame({'Term':X[0], 'Frequence':X[1]})

In [8]:
data_tf = data_tf.sort_values('Frequence', ascending = False)

In [9]:
data_tf[data_tf.Frequence >= 5].tail()

,Term,Frequence
2795,pixel,5
2420,m4,5
2586,ngoài,5
2023,gió,5
2362,lock,5


In [10]:
data_tf['length'] = data_tf['Term'].apply(len)
data_tf.head()

,Term,Frequence,length
3597,điện thoại,4141,10
3595,điện,1710,4
3091,thoại,1692,5
2926,samsung,586,7
3689,ốp,405,2


In [11]:
data_tf[['length', 'Term']].groupby('length').count()

,Term
length,
1,75
2,549
3,829
4,718
5,476
6,323
7,282
8,217
9,113


In [12]:
data_tf = data_tf.loc[(data_tf['length']>=2) & (data_tf['length'] <= 10)]

In [14]:
from underthesea import word_tokenize
user_input = 'Điện thoại sansung'
input_split = word_tokenize(user_input)
print(input_split)

['Điện thoại', 'sansung']


In [57]:
import numpy as np
def LevenshteinDistance(s, t, m = None, n = None):
    s = s.lower()
    t = t.lower()
    if m is None:
        m = len(s)
    if n is None:
        n = len(t)
    d = np.zeros(shape = (m+1, n+1))
    d[:, 0] = np.arange(m+1)
    d[0, :] = np.arange(n+1)
    
    for i in np.arange(1, m+1):
        for j in np.arange(1, n+1):
            if s[i-1] == t[j-1]:
                substitution_cost = 0
            else:
                substitution_cost = 1
            d[i, j] = min(d[i-1, j] + 1,                   # deletion
                          d[i, j-1] + 1,                   # insertion
                          d[i-1, j-1] + substitution_cost) # substitution
            
    df = pd.DataFrame(d, columns = [' '] + list(t), index = [' '] + list(s))
#     print(df)
#     ratio_length = m/n
#     if ratio_length < 1: 
#         ratio_length = 1/ratio_length
#     print(d)
#     prior words which have the length nearest with search words
#     yield (max(np.diag(d))+abs(m-n))*ratio_length
#     yield max(np.diag(d))
    return max(np.diag(d)) + abs(m-n)

In [27]:
import time
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [72]:
@timeit
def LevenshteinDistanceList(string):
    ds = [LevenshteinDistance(string, item) for item in data_tf.Term]
    idxs = np.argsort(ds)
    return data_tf.Term.iloc[idxs]

In [74]:
LevenshteinDistanceList('điện thoại')[:2]

'LevenshteinDistanceList'  9634.41 ms


3597    điện thoại
3557    Điện Thoại
Name: Term, dtype: object

In [75]:
LevenshteinDistanceList('samsung')[:2]

'LevenshteinDistanceList'  8713.19 ms


1171    SAMSUNG
2926    samsung
Name: Term, dtype: object

## Create gram

In [100]:
from nltk import bigrams, trigrams
list(bigrams(data2['Term'][1]))

[('đ', 'i'),
 ('i', 'ệ'),
 ('ệ', 'n'),
 ('n', ' '),
 (' ', 't'),
 ('t', 'h'),
 ('h', 'o'),
 ('o', 'ạ'),
 ('ạ', 'i'),
 ('i', ' '),
 (' ', 's'),
 ('s', 'a'),
 ('a', 'm'),
 ('m', 's'),
 ('s', 'u'),
 ('u', 'n'),
 ('n', 'g'),
 ('g', ' '),
 (' ', 'g'),
 ('g', 'a'),
 ('a', 'l'),
 ('l', 'a'),
 ('a', 'x'),
 ('x', 'y'),
 ('y', ' '),
 (' ', 'a'),
 ('a', '8'),
 ('8', ' '),
 (' ', 'p'),
 ('p', 'l'),
 ('l', 'u'),
 ('u', 's')]

## II. Fuzzywuzzy search

In [78]:
from fuzzywuzzy import process

@timeit
def _best_guess(input_split):
    best_guess = process.extract(input_split, data_tf.Term)
    return best_guess
_best_guess('Điện thoại')
# print(f"The best match for '{input_split[1]}' is '{best_guess}'")

'_best_guess'  2473.89 ms


[('điện thoại', 100, 3597),
 ('Điện thoại', 100, 3558),
 ('điện Thoại', 100, 3596),
 ('Điện Thoại', 100, 3557),
 ('điện', 90, 3595)]

In [79]:
process.extract('đồng hồ trẻ em', ['đog hồ trẻ em', 'do hồ trẻ em'])

[('đog hồ trẻ em', 89), ('do hồ trẻ em', 82)]

In [81]:
process.extract('đông hồ', data_tf.Term)

[('Đồng hồ', 92, 3572),
 ('đồng hồ', 92, 3658),
 ('ĐỒNG HỒ', 92, 3568),
 ('hồ', 90, 2153),
 ('đèn', 90, 3619)]

## III. Correct spelling

In [79]:
"""
Method 2 : Peter Norvig sur un seul mot
"""

import re
import nltk
from collections import Counter

def words(text): 
    return re.findall(r'\w+', text.lower())

WORDS = Counter(dic2)

In [29]:
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

P('Điện thoại')

0.0001518695137138171

In [80]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = "0123456789abcdefghijklmnopqrstuvwxyz×àáâãèéêìíòóôõùúýăđĩũơưạảấầẩẫậắằẵặẹẻẽếềểễệỉịọỏốồổỗộớờởỡợụủứừửữựỳỹ"
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

len(edits1('Điện thoại'))

2125

In [31]:
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

len(next(edits2('Điện thoại')))

11

In [32]:
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

known(['Điện thoại','điện', 'điêns'])

{'Điện thoại', 'điện'}

In [33]:
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

In [123]:
# word = 'điện thaoi'
# known([word])
# edits1(word)
# known(edits2(word))

In [34]:
@timeit
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

correction('điện thoai')

'correction'  0.00 ms


'điện thoại'

In [133]:
from underthesea import word_tokenize
"""
Exemple avec notre text initial 
"""
text1 = "đ"
text2 = "đo"
text3 = "đô"
text4 = "đôn"
text5 = "đồng"
text6 = "đồng hồ"

def common_prefix(prefix, dic = dic1):
    sub_dic = [item for item in dic1 if prefix in item]
    count = Counter(sub_dic)
    return max(count)

def correct_word_in_sentence(text):
#     tokens = word_tokenize(text)
#     r = [correction(token) for token in tokens]
    r = correction(text)
    return r

def suggest_word_search(text):
    r = correct_word_in_sentence(text)
    return common_prefix(r)

tmp = suggest_word_search(text1)
print(tmp)
tmp = suggest_word_search(text2)
print(tmp)
tmp = suggest_word_search(text3)
print(tmp)
tmp = suggest_word_search(text4)
print(tmp)
tmp = suggest_word_search(text5)
print(tmp)
tmp = suggest_word_search(text6)
print(tmp)
# tmp = ' '.join(suggest_word_search(text2))
# print(tmp)
# tmp = ' '.join(suggest_word_search(text3))
# print(tmp)
# tmp = ' '.join(suggest_word_search(text4))
# print(tmp)
# tmp = ' '.join(suggest_word_search(text5))
# print(tmp)

'correction'  0.00 ms
ưu đãi
'correction'  0.00 ms
đỡ
'correction'  0.00 ms
đông
'correction'  0.00 ms
đencủa
'correction'  0.00 ms
đồng hồ
'correction'  0.00 ms
đồng hồ


In [122]:


distribute_prefix('đồ')

'đồng hồ'